Устанавливаем необходимые библиотеки.

In [1]:
!pip install python-dotenv

  Using cached python_dotenv-0.19.2-py2.py3-none-any.whl (17 kB)


In [2]:
!pip install pymongo

  Using cached pymongo-4.0.1-cp37-cp37m-win_amd64.whl (354 kB)


In [3]:
!pip install "pymongo[srv]"

  Using cached dnspython-2.1.0-py3-none-any.whl (241 kB)


In [5]:
!pip install srsparser

  Using cached srsparser-1.0.7-py3-none-any.whl
  Using cached gensim-4.1.2-cp37-cp37m-win_amd64.whl (24.0 MB)
  Using cached nltk-3.6.7-py3-none-any.whl (1.5 MB)
  Using cached pymorphy2-0.9.1-py3-none-any.whl (55 kB)
  Using cached anytree-2.8.0-py2.py3-none-any.whl (41 kB)
  Using cached python_docx-0.8.11-py3-none-any.whl
  Using cached Cython-0.29.23-cp37-cp37m-win_amd64.whl (1.6 MB)
  Using cached smart_open-5.2.1-py3-none-any.whl (58 kB)
  Using cached scipy-1.7.3-cp37-cp37m-win_amd64.whl (34.1 MB)
  Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)
  Using cached regex-2021.11.10-cp37-cp37m-win_amd64.whl (272 kB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)
  Using cached click-8.0.3-py3-none-any.whl (97 kB)
  Using cached DAWG_Python-0.7.2-py2.py3-none-any.whl (11 kB)
  Using cached docopt-0.6.2-py2.py3-none-any.whl
  Using cached pymorphy2_dicts_ru-2.4.417127.4579844-py2.py3-none-any.whl (8.2 MB)


Подключаемся к базе данных с "распаршенными" текстовыми документами с ТЗ на MongoDB.

In [6]:
from pymongo import MongoClient
from dotenv import load_dotenv
import os

load_dotenv()

# строка подключения к кластеру MongoDB
CONNECTION_STRING = os.getenv('CONNECTION_STRING')

# название базы данных с результатами анализа текстовых документов
DB_NAME = "documentsAnalysis"
# название коллекции с результатами анализа текстовых документов
TMPL_COLL_NAME = "requirementsSpecifications"

# подключение к MongoDB
client = MongoClient(CONNECTION_STRING)
# подключение к БД на MongoDB
db = client[DB_NAME]
# подключение к коллекции, в которой находятся результаты анализа текстовых документов
coll = db[TMPL_COLL_NAME]
# взятие оттуда всех результатов
results = list(coll.find({}))

"Распаршенные" текстовые документы, где каждый отдельно взятый результат содержит название текстового документа с ТЗ и заполненную информацией структуру дерева разделов.

In [9]:
for result in results:
    print("Название документа:", result["document_name"], "\nРезультирующая структура:", result["structure"], end="\n\n")

Название документа: tz_01 
Результирующая структура: {'name': 'Техническое задание', 'children': [{'text': 'полное наименование системы и ее условное обозначение: Конфигурация «Бухгалтерия предприятия» в среде «1С: Предприятие 8.1»; \nшифр темы: ДП – УлГТУ –  08080165 – 00/00000 – 2010;\nнаименование предприятий разработчика и заказчика системы, их реквизиты: УМКУП «Городской градостроительный сервис», .Ульяновск, пер.Комсомольский 3, 8-962-633-01-77;\nперечень документов, на основании которых создается информационная система: ; \nплановые сроки начала и окончания работ:1 апреля . – 15 июня .;\nсведения об источниках и порядке финансирования работ: работа проводится на безвозмездной основе, не финансируется;\nпорядок оформления и предъявления заказчику результатов работ по созданию системы, ее частей и отдельных средств: по окончанию работ заказчику передается установочный диск со всеми дистрибутивами, необходимыми для функционирования разработанного программного продукта, а так же соп

Используя функцию get_tf_idf_weights из пакета srsparser, построим для каждого документа по всему тексту модель TF-IDF.

In [12]:
from srsparser import get_tf_idf_weights

# вытаскиваем из каждого резульата структуру дерева разделов
structs = []
for result in results:
    structs.append(result["structure"])

# для текстового содержания каждой структуры получаем TF-IDF веса слов
for struct_tf_idf_weights in get_tf_idf_weights(structs):
    print(struct_tf_idf_weights, end="\n\n") # вывод результатов в том же порядке, в каком результаты лежат в коллекции MongoDB

[['celeron', 0.021], ['ii', 0.038], ['iii', 0.033], ['intel', 0.045], ['iv', 0.043], ['microsoft', 0.072], ['ms', 0.033], ['pack', 0.119], ['pentium', 0.019], ['server', 0.055], ['service', 0.119], ['sql', 0.084], ['vista', 0.063], ['windows', 0.008], ['xeon', 0.028], ['xp', 0.024], ['авариин', 0.046], ['автоматизация', 0.012], ['автоматизировать', 0.019], ['авторизовать', 0.059], ['академический', 0.038], ['анализ', 0.005], ['аналитический', 0.046], ['аппаратноить', 0.046], ['аппаратный', 0.032], ['апрель', 0.019], ['база', 0.014], ['базовой', 0.119], ['башаровой', 0.059], ['безвозмездноить', 0.019], ['бухгалтер', 0.038], ['бухгалтерия', 0.084], ['бухгалтерский', 0.174], ['быстрый', 0.032], ['ввод', 0.022], ['вводить', 0.046], ['версия', 0.032], ['вести', 0.032], ['весь', 0.014], ['взаимодеиствие', 0.013], ['вид', 0.001], ['включать', 0.013], ['владение', 0.046], ['влиять', 0.059], ['внештатный', 0.046], ['возвращаться', 0.046], ['возлагаться', 0.092], ['возможно', 0.038], ['возможнос

Используя функцию get_tf_idf_weights из пакета srsparser, построим для каждого документа по разделу функциональные требования модель TF-IDF (Пустые массивы в выводе свидетельствуют о том, что в структуре не было раздела "Требования к функциям (задачам)").

In [13]:
# для текстового содержания раздела "Требования к функциям (задачам)" каждой структуры получаем TF-IDF веса слов
for struct_tf_idf_weights in get_tf_idf_weights(structs, "Требования к функциям (задачам)"):
    print(struct_tf_idf_weights, end="\n\n") # вывод результатов в том же порядке, в каком результаты лежат в коллекции MongoDB

[['аналитический', 0.092], ['бухгалтерский', 0.305], ['ввод', 0.025], ['вести', 0.119], ['возможно', 0.119], ['входящий', 0.092], ['данные', 0.048], ['добавление', 0.092], ['документ', 0.126], ['должный', 0.087], ['другой', 0.038], ['заработной', 0.092], ['иметь', 0.13], ['импорт', 0.119], ['информация', 0.016], ['итоговой', 0.119], ['лишь', 0.119], ['начисление', 0.092], ['новый', 0.092], ['обладать', 0.119], ['объект', 0.119], ['оперативный', 0.305], ['операция', 0.065], ['ориентированный', 0.119], ['основа', 0.238], ['основной', 0.119], ['осуществлять', 0.119], ['относиться', 0.038], ['отчёт', 0.13], ['первичный', 0.119], ['плата', 0.092], ['подготовка', 0.119], ['подсистема', 0.289], ['поиск', 0.092], ['постоянный', 0.119], ['произвольный', 0.119], ['просмотр', 0.13], ['процесс', 0.043], ['работа', 0.019], ['расчёт', 0.076], ['регистрация', 0.065], ['редактирование', 0.065], ['система', 0.075], ['следующей', 0.092], ['случай', 0.119], ['соответствующей', 0.092], ['сортировка', 0.11

Используя функцию get_tf_idf_weights из пакета srsparser, построим для каждого документа по всему тексту модель TF-IDF, но теперь только для СУЩЕСТВИТЕЛЬНЫХ.

In [14]:
# для текстового содержания каждой структуры получаем TF-IDF веса слов, исключив из текста все слова, кроме существительных
for struct_tf_idf_weights in get_tf_idf_weights(structs, part_of_speech="NOUN"):
    print(struct_tf_idf_weights, end="\n\n") # вывод результатов в том же порядке, в каком результаты лежат в коллекции MongoDB

[['авариин', 0.069], ['автоматизация', 0.019], ['анализ', 0.008], ['апрель', 0.028], ['база', 0.021], ['башаровой', 0.089], ['бухгалтер', 0.057], ['бухгалтерия', 0.126], ['ввод', 0.033], ['версия', 0.049], ['взаимодеиствие', 0.019], ['вид', 0.001], ['владение', 0.069], ['возможность', 0.009], ['возникновение', 0.049], ['восстановление', 0.171], ['вывод', 0.025], ['выполнение', 0.008], ['главное', 0.069], ['группа', 0.069], ['деиствие', 0.012], ['деиствия', 0.057], ['деятельность', 0.024], ['диск', 0.036], ['дистрибутив', 0.025], ['добавление', 0.028], ['документ', 0.01], ['документация', 0.026], ['дополнение', 0.089], ['достаточность', 0.089], ['достоверность', 0.049], ['доступ', 0.063], ['доцент', 0.032], ['драивер', 0.069], ['жесткия', 0.114], ['загрузка', 0.042], ['задание', 0.014], ['задача', 0.016], ['заказчик', 0.012], ['замена', 0.089], ['заполнение', 0.028], ['запрашиваемои', 0.089], ['защита', 0.05], ['значение', 0.028], ['значок', 0.069], ['идентификация', 0.089], ['изменение

Используя функцию get_tf_idf_weights из пакета srsparser, построим для каждого документа по разделу функциональные требования модель TF-IDF (Пустые массивы в выводе свидетельствуют о том, что в структуре не было раздела "Требования к функциям (задачам)"), но теперь только для СУЩЕСТВИТЕЛЬНЫХ.

In [15]:
# для текстового содержания раздела "Требования к функциям (задачам)" каждой структуры получаем TF-IDF веса слов, исключив из текста все слова, кроме существительных
for struct_tf_idf_weights in get_tf_idf_weights(structs, "Требования к функциям (задачам)", "NOUN"):
    print(struct_tf_idf_weights, end="\n\n") # вывод результатов в том же порядке, в каком результаты лежат в коллекции MongoDB

[['ввод', 0.036], ['добавление', 0.129], ['документ', 0.178], ['импорт', 0.167], ['информация', 0.022], ['начисление', 0.129], ['объект', 0.167], ['операция', 0.091], ['основа', 0.335], ['отчёт', 0.182], ['плата', 0.129], ['подготовка', 0.167], ['подсистема', 0.407], ['поиск', 0.129], ['просмотр', 0.182], ['процесс', 0.06], ['работа', 0.026], ['расчёт', 0.107], ['регистрация', 0.091], ['редактирование', 0.091], ['система', 0.105], ['случай', 0.167], ['сортировка', 0.167], ['состав', 0.091], ['сотрудник', 0.091], ['список', 0.214], ['средство', 0.047], ['сумма', 0.107], ['удаление', 0.258], ['учёт', 0.241], ['фильтрация', 0.167], ['формирование', 0.106], ['функциональность', 0.167], ['функция', 0.026], ['хранение', 0.121], ['экспорт', 0.167], ['элемент', 0.229]]

[['отчёт', 0.111], ['регистрация', 0.333], ['система', 0.05], ['сотрудник', 0.111], ['учёт', 0.074], ['формирование', 0.065], ['автоматизация', 0.074], ['ведение', 0.13], ['выдача', 0.315], ['выполнение', 0.128], ['выработка', 